## Generate vacuum r-htf for THR->ALA (flattened torsions/exceptions) @ lambda = 0 endstate

In [ ]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [ ]:
# Generate htf for capped THR->ALA in vacuum
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs=None,
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}, 
                                   molecules=None, 
                                   cache=None)

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(pdb.topology)

htf = generate_dipeptide_top_pos_sys(pdb.topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

In [ ]:
import os
out_dir = ''
pickle.dump(htf, open(os.path.join(out_dir, ".pickle"), "wb" ))


## Create alchemical system

In [ ]:
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import pickle
import argparse
import os



In [ ]:
# Read htf
out_dir = ''
with open(os.path.join(out_dir, f".pickle"), "rb") as f:
    htf = pickle.load(f)


In [ ]:
endstate = 0

# Alchemify the hybrid system
if endstate == 0: # Alchemical region = mutated residue atoms only
    atoms_to_alchemify = list(htf._atom_classes['unique_old_atoms']) + [6, 7, 8, 9, 13, 14]
elif endstate == 1:
    atoms_to_alchemify = list(htf._atom_classes['unique_new_atoms']) + [6, 7, 8, 9, 13, 14]

alch_factory = AbsoluteAlchemicalFactory(consistent_exceptions=False)
alchemical_region = AlchemicalRegion(alchemical_atoms=list(atoms_to_alchemify), alchemical_torsions=True, annihilate_sterics=True, annihilate_electrostatics=True)
alchemical_system = alch_factory.create_alchemical_system(htf.hybrid_system, alchemical_region)

# Initialize compound thermodynamic states at different temperatures and alchemical states.
protocol = {'temperature': [300]*unit.kelvin*11,
            'lambda_electrostatics': [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0],
            'lambda_sterics': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
           'lambda_torsions': [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]}

alchemical_state = AlchemicalState.from_system(alchemical_system)
compound_states = states.create_thermodynamic_state_protocol(alchemical_system, 
                                                             protocol=protocol, 
                                                             composable_states=[alchemical_state])